In [11]:
import pandas as pd
from sklearn.utils import shuffle

In [4]:
df_data = pd.read_csv("../../results/processed_data/full_dataset_with_negative_examples.csv")
df_data

,id,sequence,target
0,sp|P79073|HYP2_HYPJE,MQFFAVALFATSALAAVCPTGLFSNPLCCATNVLDLIGVDCKTPTI...,HFB
1,sp|Q04571|RODL_NEUCR,MQFTSVFTILAIAMTAAAAPAEVVPRATTIGPNTCSIDDYKPYCCQ...,HFB
2,sp|Q9UVI4|THYD_CLAFS,MKFLAAASLLVASTLAVPTSSGGSCRPRPPPGGGNGGNGGNGGNGG...,HFB
3,sp|D4ARW0|HYPA_ARTBC,MKFSIAAAVLALASAVVAHPGAGYVSTPEKEANFQQNFQKFVTACG...,HFB
4,sp|O13300|HYP3_AGABI,MFARISTIITTLFFAMLAAATAVPRTDPPPATGSQCTAVGGDVNCC...,HFB
...,...,...,...
42226,sp|A5USH6|RS14Z_ROSS1,MARKALIVKAQRPQKYKVRAYNRCKICGRPRAYMRKFGMCRICFRE...,Negative
42227,sp|A4WDA8|HSCB_ENT38,MDYFTLFGLPAQYSIDLPALTIRFQDLQRQFHPDKFASGTPAEQLA...,Negative
42228,sp|Q0I8J6|PURA_SYNS3,MANVVVIGAQWGDEGKGKITDLLSRSADVVVRYQGGVNAGHTIVVD...,Negative
42229,sp|A9NBQ9|TRMD_COXBR,MKLIIGVITLFPQMFDALKSGVIGRALKQDRLTLSFWNPRDYATDP...,Negative


### Generating dataset with binary examples

In [7]:
df_data_pos_neg = df_data[["sequence", "target"]]
df_data_pos = df_data_pos_neg[df_data["target"] != "Negative"]
df_data_neg = df_data_pos_neg[df_data["target"] == "Negative"]

df_data_pos = df_data_pos[["sequence"]]
df_data_pos["target"] = "Positive"

df_data_pos_neg = pd.concat([df_data_pos, df_data_neg], axis=0)
df_data_pos_neg.to_csv("../../results/datasets_to_code/binary_case_data.csv", index=False)
df_data_pos_neg['target'].value_counts()


target
Negative    21126
Positive    21105
Name: count, dtype: int64

### Generating multiclass dataset with negative examples

In [8]:
df_data['target'].value_counts()

target
Negative        21126
HFB             12034
HFB Class I      6698
CPs              1610
HFB Class II      763
Name: count, dtype: int64

In [9]:
df_data_neg = df_data[df_data['target'] == "Negative"]
df_hfb = df_data[df_data['target'] == "HFB"]
df_hfb_c1 = df_data[df_data['target'] == "HFB Class I"]
df_hfb_c2 = df_data[df_data['target'] == "HFB Class II"]
df_cps = df_data[df_data['target'] == "CPs"]


In [12]:
df_data_neg = shuffle(df_data_neg)
df_hfb = shuffle(df_hfb)
df_hfb_c1 = shuffle(df_hfb_c1)
df_cps = shuffle(df_cps)

In [13]:
df_data_neg = df_data_neg[:len(df_hfb_c2)]
df_hfb = df_hfb[:len(df_hfb_c2)]
df_hfb_c1 = df_hfb_c1[:len(df_hfb_c2)]
df_cps = df_cps[:len(df_hfb_c2)]

df_concat_multi_class = pd.concat([df_data_neg,df_hfb, df_hfb_c1, df_cps, df_hfb_c2 ], axis=0)
df_concat_multi_class['target'].value_counts()

target
Negative        763
HFB             763
HFB Class I     763
CPs             763
HFB Class II    763
Name: count, dtype: int64

In [16]:
df_concat_multi_class = df_concat_multi_class[["sequence", "target"]]
df_concat_multi_class.to_csv("../../results/datasets_to_code/multi_class_with_negative.csv", index=False)

### Generating dataset binary with only positive examples HFB v/s CPs

In [17]:
df_data_pos = df_data[["sequence", "target"]]
df_data_pos = df_data_pos[df_data_pos["target"] != "Negative"]
df_data_pos["target"].value_counts()

target
HFB             12034
HFB Class I      6698
CPs              1610
HFB Class II      763
Name: count, dtype: int64

In [20]:
df_hfb = df_data_pos[df_data_pos["target"] == "HFB"]
df_CPs = df_data_pos[df_data_pos["target"] == "CPs"]

df_hfb = shuffle(df_hfb)

df_concat = pd.concat([df_CPs, df_hfb[:len(df_CPs)]], axis=0)

df_concat.to_csv("../../results/datasets_to_code/binary_only_positive.csv", index=False)
df_concat["target"].value_counts()


target
CPs    1610
HFB    1610
Name: count, dtype: int64

### Generating only HFBs multitask

In [21]:
df_data_pos = df_data[["sequence", "target"]]
df_data_hfb = df_data_pos[df_data_pos["target"] == "HFB"]
df_data_hfb_c1 = df_data_pos[df_data_pos["target"] == "HFB Class I"]
df_data_hfb_c2 = df_data_pos[df_data_pos["target"] == "HFB Class II"]

In [22]:
unique_hfb_c1 = df_data_hfb_c1['sequence'].unique().tolist()
unique_hfb_c2 = df_data_hfb_c2['sequence'].unique().tolist()

is_in_c1_c2 = []

for index in df_data_hfb.index:
    seq = df_data_hfb['sequence'][index]

    if seq not in unique_hfb_c1 and seq not in unique_hfb_c2:
        is_in_c1_c2.append(False)
    else:
        is_in_c1_c2.append(True)

df_data_hfb['is_in_c1_c2'] = is_in_c1_c2
df_data_hfb

/tmp/ipykernel_104767/2990527178.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_hfb['is_in_c1_c2'] = is_in_c1_c2


,sequence,target,is_in_c1_c2
0,MQFFAVALFATSALAAVCPTGLFSNPLCCATNVLDLIGVDCKTPTI...,HFB,False
1,MQFTSVFTILAIAMTAAAAPAEVVPRATTIGPNTCSIDDYKPYCCQ...,HFB,False
2,MKFLAAASLLVASTLAVPTSSGGSCRPRPPPGGGNGGNGGNGGNGG...,HFB,True
3,MKFSIAAAVLALASAVVAHPGAGYVSTPEKEANFQQNFQKFVTACG...,HFB,True
4,MFARISTIITTLFFAMLAAATAVPRTDPPPATGSQCTAVGGDVNCC...,HFB,True
...,...,...,...
12029,MNTNVTAPAPSTIPINQEHTVQQHKDTIPAYTSAEFDNPQQQYTPA...,HFB,False
12030,MLFNNIIVAIALSGAATALPQKYPIKCGDSVCPADKPKCCEVLVNG...,HFB,False
12031,MQFSLAIVTLLATAVSALPTEEKRQAYIPCSGLYGTSQCCATDVLG...,HFB,False
12032,MKTTALLSALTLAVVAIAAPAEIEARGRGSGGGGSGGTCNSDNHQQ...,HFB,False


In [24]:
df_data_hfb = df_data_hfb[df_data_hfb["is_in_c1_c2"] ==False]
df_data_hfb = df_data_hfb[["sequence", "target"]]
df_data_hfb

,sequence,target
0,MQFFAVALFATSALAAVCPTGLFSNPLCCATNVLDLIGVDCKTPTI...,HFB
1,MQFTSVFTILAIAMTAAAAPAEVVPRATTIGPNTCSIDDYKPYCCQ...,HFB
7,MRFIVSLLAFTAAATATALPASAAKNAKLATSAAFAKQAEGTTCNV...,HFB
9,MKFFAIAALFAAAAVAQPLEDRSNGNGNVCPPGLFSNPQCCATQVL...,HFB
10,MQFSAAAITAIAMALTVTAAPGTTAPVKRTLDTLTYAEAQNVCGQN...,HFB
...,...,...
12029,MNTNVTAPAPSTIPINQEHTVQQHKDTIPAYTSAEFDNPQQQYTPA...,HFB
12030,MLFNNIIVAIALSGAATALPQKYPIKCGDSVCPADKPKCCEVLVNG...,HFB
12031,MQFSLAIVTLLATAVSALPTEEKRQAYIPCSGLYGTSQCCATDVLG...,HFB
12032,MKTTALLSALTLAVVAIAAPAEIEARGRGSGGGGSGGTCNSDNHQQ...,HFB


In [28]:
df_data_hfb = shuffle(df_data_hfb)
df_data_hfb_c1 = shuffle(df_data_hfb_c1)

df_concat_hfb_class = pd.concat([df_data_hfb_c2, df_data_hfb[:len(df_data_hfb_c2)], df_data_hfb_c1[:len(df_data_hfb_c2)]], axis=0)

df_concat_hfb_class.to_csv("../../results/datasets_to_code/multi_task_HFB.csv", index=False)
df_concat_hfb_class["target"].value_counts()

target
HFB Class II    763
HFB             763
HFB Class I     763
Name: count, dtype: int64